In [ ]:
import pickle
from pathlib import Path
import warnings

import pandas as pd
import networkx as nx
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.exceptions import ConvergenceWarning
from dowhy import CausalModel

from structure_data import choose_columns, preprocess_data

In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
ROOT = Path("./")
RAW_DATA = ROOT / "raw_data/csv/eqls_2007and2011.csv"
DICT_PATH = ROOT / "data/dictionary.json"
GRAPH_PATH = ROOT / "graphs/full_causal.gpickle"
TREATMENT = "Y11_Q57"
OUTCOME = "Y11_MWIndex"


def _prep_ordinals_inplace(df: pd.DataFrame, ord_cols, known_orders=None):
    """
    Map ordinal levels to 0..m-1 and scale to [0,1] so L1 equals Gower per-feature.
    known_orders: optional dict {col: [lowest,...,highest]} to control ordering.
    """
    for c in ord_cols:
        if known_orders and c in known_orders:
            order = {v: i for i, v in enumerate(known_orders[c])}
            df[c] = df[c].map(order).astype(float)
            m = max(order.values()) if order else 1
            df[c] = df[c] / (m if m > 0 else 1.0)
        else:
            # infer: if numeric-coded, scale min–max; else sort unique labels
            if pd.api.types.is_numeric_dtype(df[c]):
                lo, hi = df[c].min(), df[c].max()
                rng = (hi - lo) if hi > lo else 1.0
                df[c] = (df[c] - lo) / rng
            else:
                levels = sorted(df[c].dropna().unique().tolist())
                mapping = {v: i for i, v in enumerate(levels)}
                m = max(mapping.values()) if mapping else 1
                df[c] = df[c].map(mapping).astype(float) / (m if m > 0 else 1.0)


def load_data() -> pd.DataFrame:
    """Load and preprocess the raw EQLS data."""
    bdvs = [
        "Y11_EmploymentStatus",
        "Y11_HHstructure",
        "Y11_HHsize",
        "Y11_Agecategory",
        "Y11_Q7",
        "Y11_Q31",
        "Y11_Country",
        "Y11_Q32",
        "Y11_HH2a",
        TREATMENT,
        OUTCOME,
    ]
    df = choose_columns()
    df = preprocess_data(
        df,
        na_threshold=0.5,
        impute_strategy="drop",
        treatment_dichotomize_value="median",
        treatment_column=TREATMENT,
        backdoor_variables=bdvs,
    )
    df.to_csv("data/eqls_processed.csv", index=False)
    return df


def get_schema() -> dict:
    categorical = ["Y11_Q32", "Y11_Q7"]
    ordinal = [
        "Y11_Agecategory",
        "Y11_Country",
        "Y11_EmploymentStatus",
        "Y11_HH2a",
        "Y11_HHsize",
        "Y11_HHstructure",
        "Y11_Q31",
    ]
    return {"cat": categorical, "ord": ordinal}


def load_graph() -> nx.DiGraph:
    """Load the causal graph describing relationships among variables."""
    with open(GRAPH_PATH, "rb") as f:
        return pickle.load(f)


def t_learner_estimate(
    df: pd.DataFrame,
    schema: dict,
    treatment_col: str,
    outcome_col: str,
) -> float:
    """
    T-learner ATE using two outcome models:
      - f1 trained on treated data
      - f0 trained on control data
    Returns mean_i [ f1(x_i) - f0(x_i) ].
    If outcome is not binary {0,1}, falls back to linear regression.

    Assumes df already has ordinal columns numerically scaled.
    """
    # Features: confounders only (no T, no Y)
    feat_cols = schema["cat"] + schema["ord"]
    X = df[feat_cols].copy()

    # Ensure categoricals are treated as such, then one-hot on FULL X (to align cols)
    for c in schema["cat"]:
        X[c] = X[c].astype("category")
    X_enc = pd.get_dummies(X, columns=schema["cat"], drop_first=True)

    T = df[treatment_col].astype(int).values
    y = df[outcome_col].values

    X1, y1 = X_enc[T == 1], y[T == 1]
    X0, y0 = X_enc[T == 0], y[T == 0]

    g1 = LinearRegression()
    g0 = LinearRegression()
    g1.fit(X1, y1)
    g0.fit(X0, y0)
    mu1 = g1.predict(X_enc)
    mu0 = g0.predict(X_enc)

    ite = mu1 - mu0
    ate = float(np.nanmean(ite))
    return ate


def estimate_effects(df: pd.DataFrame, graph) -> dict:
    df2 = df.copy()
    _prep_ordinals_inplace(df2, get_schema()["ord"])

    model = CausalModel(
        data=df2,
        treatment=TREATMENT,
        outcome=OUTCOME,
        graph=nx.nx_pydot.to_pydot(graph).to_string(),
    )
    estimand = model.identify_effect()

    methods = [
        "backdoor.propensity_score_matching",
        "backdoor.propensity_score_weighting",
        "backdoor.propensity_score_stratification",
        "backdoor.linear_regression",
        "backdoor.distance_matching",
        "backdoor.T_learner",  # <- custom
    ]
    kwargs = {
        "backdoor.distance_matching": dict(
            target_units="ate",
            method_params={
                "distance_metric": "minkowski",
                "p": 1,
                "num_matches_per_unit": 1,
                "exact_match_cols": get_schema()["cat"],
            },
        )
    }

    results = {}
    for m in methods:
        try:
            if m == "backdoor.T_learner":
                # Call our custom estimator
                ate = t_learner_estimate(
                    df2, get_schema(), treatment_col=TREATMENT, outcome_col=OUTCOME
                )
                results[m] = ate
            else:
                est = model.estimate_effect(
                    estimand, method_name=m, **kwargs.get(m, {})
                )
                results[m] = float(est.value)
        except Exception as e:
            print(f"[!] Estimation with {m} failed: {e}")
            results[m] = float("nan")
    return results

In [ ]:
df = load_data()
graph = load_graph()
results = estimate_effects(df, graph)

print("Estimation results (ATE):")
for name, val in results.items():
    print(f"  {name:<40} {val:.4f}")